In [2]:
from platform import python_version
import pkg_resources
import pickle
import numpy as np
import pandas as pd
import sklearn


In [3]:
cgm= pd.read_csv("CGMData.csv")
pump= pd.read_csv("InsulinData.csv")

C:\Users\15713\AppData\Local\Temp\ipykernel_10356\3399335140.py:1: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  cgm= pd.read_csv("CGMData.csv")
C:\Users\15713\AppData\Local\Temp\ipykernel_10356\3399335140.py:2: DtypeWarning: Columns (13,43) have mixed types. Specify dtype option on import or set low_memory=False.
  pump= pd.read_csv("InsulinData.csv")


In [5]:
print(cgm)

       Index       Date      Time  New Device Time  BG Reading (mg/dL)  \
0      20355  2/12/2018  13:22:27              NaN                 NaN   
1      20356  2/12/2018  13:17:27              NaN                 NaN   
2      20357  2/12/2018  13:12:27              NaN                 NaN   
3      20358  2/12/2018  13:07:27              NaN                 NaN   
4      20359  2/12/2018  13:02:27              NaN                 NaN   
...      ...        ...       ...              ...                 ...   
55338  52855  7/25/2017  12:28:54              NaN                 NaN   
55339  52856  7/25/2017  12:23:54              NaN                 NaN   
55340  52857  7/25/2017  12:18:54              NaN                 NaN   
55341  52858  7/25/2017  12:13:54              NaN                 NaN   
55342  52859  7/25/2017  12:08:54              NaN                 NaN   

       Linked BG Meter ID  Basal Rate (U/h)  Temp Basal Amount  \
0                     NaN               NaN  

In [6]:
cgm[['Date','Time','Sensor Glucose (mg/dL)']]

,Date,Time,Sensor Glucose (mg/dL)
0,2/12/2018,13:22:27,118.0
1,2/12/2018,13:17:27,122.0
2,2/12/2018,13:12:27,NaN
3,2/12/2018,13:07:27,NaN
4,2/12/2018,13:02:27,NaN
...,...,...,...
55338,7/25/2017,12:28:54,311.0
55339,7/25/2017,12:23:54,311.0
55340,7/25/2017,12:18:54,309.0
55341,7/25/2017,12:13:54,310.0


In [13]:
pump=pump[['Date','Time','Alarm']]

In [8]:
''' Tasks
1. Percentage time in hyperglycemia (CGM > 180 mg/dL)

2. Percentage of time in hyperglycemia critical (CGM > 250 mg/dL)

3. Percentage time in range (CGM >= 70 mg/dL and CGM <= 180 mg/dL)

4. Percentage time in range secondary (CGM >= 70 mg/dL and CGM <= 150 mg/dL)

5. Percentage time in hypoglycemia level 1 (CGM < 70 mg/dL)

6. Percentage time in hypoglycemia level 2 (CGM < 54 mg/dL)
'''


' Tasks\n1. Percentage time in hyperglycemia (CGM > 180 mg/dL)\n\n2. Percentage of time in hyperglycemia critical (CGM > 250 mg/dL)\n\n3. Percentage time in range (CGM >= 70 mg/dL and CGM <= 180 mg/dL)\n\n4. Percentage time in range secondary (CGM >= 70 mg/dL and CGM <= 150 mg/dL)\n\n5. Percentage time in hypoglycemia level 1 (CGM < 70 mg/dL)\n\n6. Percentage time in hypoglycemia level 2 (CGM < 54 mg/dL)\n'

In [14]:
pump[pump['Alarm'] == "AUTO MODE ACTIVE PLGM OFF"]

,Date,Time,Alarm
20449,11/4/2017,17:36:46,AUTO MODE ACTIVE PLGM OFF
40131,8/9/2017,8:07:13,AUTO MODE ACTIVE PLGM OFF


In [15]:
sw= pump[pump['Alarm'] == "AUTO MODE ACTIVE PLGM OFF"][['Date','Time']].iloc[-1]

In [16]:
sw['Time']

'8:07:13'

In [43]:
ind= cgm[(cgm['Date'] == sw['Date']) & (pd.to_datetime(cgm['Time'],format='%H:%M:%S').dt.time <= pd.to_datetime(sw['Time'],format='%H:%M:%S').time())][['Date','Time','Alarm']].index[0]
ind

51087

In [44]:
autoData= cgm.iloc[0:ind]
manualData= cgm.iloc[ind:len(cgm)]

In [45]:
len(autoData)

51087

In [46]:
len(manualData)

4256

In [47]:
mDays = manualData["Date"].unique()[::-1]

In [48]:
manualHyperglycemiaCriticalDay, manualHyperglycemiaCriticalMorning, manualHyperglycemiaCriticalNight= 0, 0, 0
manualHyperglycemiaDay, manualHyperglycemiaMorning, manualHyperglycemiaNight= 0, 0, 0
manualRangeDay, manualRangeMorning, manualRangeNight= 0, 0, 0
manualRangeSecondaryDay, manualRangeSecondaryMorning, manualRangeSecondaryNight= 0, 0, 0
manualHypoglycemia1Day, manualHypoglycemia1Morning, manualHypoglycemia1Night= 0, 0, 0
manualHypoglycemia2Day, manualHypoglycemia2Morning, manualHypoglycemia2Night= 0, 0, 0
totalManual= 0
totalManualMorning= 0
totalManualNight= 0

for d in mDays:
    #print(f"Day:{d}, Rows:{len(manualData[(manualData['Date'] == d) & (manualData['Sensor Glucose (mg/dL)'].notna())])}")
    actualNight= len(manualData[(manualData['Date'] == d) & (pd.to_datetime(manualData['Time'],format='%H:%M:%S').dt.time < pd.to_datetime("6:00:00",format='%H:%M:%S').time()) & (manualData['Sensor Glucose (mg/dL)'].notna())])
    totalNight= len(manualData[(manualData['Date'] == d) & (pd.to_datetime(manualData['Time'],format='%H:%M:%S').dt.time < pd.to_datetime("6:00:00",format='%H:%M:%S').time())])
    nightDiff= totalNight - actualNight
    
    actualMorning= len(manualData[(manualData['Date'] == d) & (pd.to_datetime(manualData['Time'],format='%H:%M:%S').dt.time >= pd.to_datetime("6:00:00",format='%H:%M:%S').time()) & (manualData['Sensor Glucose (mg/dL)'].notna())])
    totalMorning= len(manualData[(manualData['Date'] == d) & (pd.to_datetime(manualData['Time'],format='%H:%M:%S').dt.time >= pd.to_datetime("6:00:00",format='%H:%M:%S').time())])
    morningDiff= totalMorning - actualMorning
    
    actualDay= len(manualData[(manualData['Date'] == d) & (manualData['Sensor Glucose (mg/dL)'].notna())])
    totalDay= len(manualData[(manualData['Date'] == d)])
    dayDiff= totalDay - actualDay
        
    if dayDiff <= 10 or totalDay == 288:
        cNum = 1
        
        hyperglycemiaCriticalDay, hyperglycemiaCriticalMorning, hyperglycemiaCriticalNight= 0, 0, 0
        hyperglycemiaDay, hyperglycemiaMorning, hyperglycemiaNight= 0, 0, 0
        rangeDay, rangeMorning, rangeNight= 0, 0, 0
        rangeSecondaryDay, rangeSecondaryMorning, rangeSecondaryNight= 0, 0, 0
        hypoglycemia1Day, hypoglycemia1Morning, hypoglycemia1Night= 0, 0, 0
        hypoglycemia2Day, hypoglycemia2Morning, hypoglycemia2Night= 0, 0, 0
        
        data = manualData[(manualData['Date'] == d)].interpolate(limit=10)
        for l in data['Sensor Glucose (mg/dL)']:
            if l > 250:
                hyperglycemiaCriticalDay += 1
                if cNum <= 216: hyperglycemiaCriticalMorning += 1  
                else: hyperglycemiaCriticalNight += 1
            if l <= 250 and l > 180:
                hyperglycemiaDay += 1
                if cNum <= 216: hyperglycemiaMorning += 1 
                else: hyperglycemiaNight += 1
            if l <= 180 and l >= 70:
                rangeDay += 1
                if cNum <= 216: rangeMorning += 1
                else: rangeNight += 1
            if l <= 150 and l >= 70:
                rangeSecondaryDay += 1
                if cNum <= 216: rangeSecondaryMorning += 1
                else: rangeSecondaryNight += 1
            if l < 70 and l >= 54:
                hypoglycemia1Day += 1
                if cNum <= 216: hypoglycemia1Morning += 1
                else: hypoglycemia1Night += 1
            if l < 54:
                hypoglycemia2Day += 1
                if cNum <= 216: hypoglycemia2Morning += 1
                else: hypoglycemia2Night += 1
            cNum += 1
        manualHyperglycemiaCriticalDay += hyperglycemiaCriticalDay/288
        manualHyperglycemiaCriticalMorning += hyperglycemiaCriticalMorning/216
        manualHyperglycemiaCriticalNight += hyperglycemiaCriticalNight/72
        manualHyperglycemiaDay += hyperglycemiaDay/288
        manualHyperglycemiaMorning += hyperglycemiaMorning/216
        manualHyperglycemiaNight += hyperglycemiaNight/72
        manualRangeDay += rangeDay/288
        manualRangeMorning += rangeMorning/216
        manualRangeNight += rangeNight/72
        manualRangeSecondaryDay += rangeSecondaryDay/288
        manualRangeSecondaryMorning += rangeSecondaryMorning/216
        manualRangeSecondaryNight += rangeSecondaryNight/72
        manualHypoglycemia1Day += hypoglycemia1Day/288
        manualHypoglycemia1Morning += hypoglycemia1Morning/216
        manualHypoglycemia1Night += hypoglycemia1Night/72
        manualHypoglycemia2Day += hypoglycemia2Day/288
        manualHypoglycemia2Morning += hypoglycemia2Morning/216
        manualHypoglycemia2Night += hypoglycemia2Night/72
        totalManual += 1
        totalManualMorning += 1
        totalManualNight += 1
    elif morningDiff <= 10 or totalMorning == 216:
        cNum = 1
        
        hyperglycemiaCriticalDay, hyperglycemiaCriticalMorning, hyperglycemiaCriticalNight= 0, 0, 0
        hyperglycemiaDay, hyperglycemiaMorning, hyperglycemiaNight= 0, 0, 0
        rangeDay, rangeMorning, rangeNight= 0, 0, 0
        rangeSecondaryDay, rangeSecondaryMorning, rangeSecondaryNight= 0, 0, 0
        hypoglycemia1Day, hypoglycemia1Morning, hypoglycemia1Night= 0, 0, 0
        hypoglycemia2Day, hypoglycemia2Morning, hypoglycemia2Night= 0, 0, 0
        
        data= manualData[(manualData['Date'] == d)& (pd.to_datetime(manualData['Time'],format='%H:%M:%S').dt.time >= pd.to_datetime("6:00:00",format='%H:%M:%S').time())].interpolate(limit=10)
        for l in data['Sensor Glucose (mg/dL)']:
            if l > 250:
                hyperglycemiaCriticalMorning += 1  
            if l <= 250 and l > 180:
                hyperglycemiaMorning += 1 
            if l <= 180 and l >= 70:
                rangeMorning += 1
            if l <= 150 and l >= 70:
                rangeSecondaryMorning += 1
            if l < 70 and l >= 54:
                hypoglycemia1Morning += 1
            if l < 54:
                hypoglycemia2Morning += 1
            cNum += 1
        manualHyperglycemiaCriticalMorning += hyperglycemiaCriticalMorning/216
        manualHyperglycemiaMorning += hyperglycemiaMorning/216
        manualRangeMorning += rangeMorning/216
        manualRangeSecondaryMorning += rangeSecondaryMorning/216
        manualHypoglycemia1Morning += hypoglycemia1Morning/216
        manualHypoglycemia2Morning += hypoglycemia2Morning/216
        totalManualMorning += 1
    elif nightDiff <= 10 or totalNight == 72:
        cNum = 1
        
        hyperglycemiaCriticalDay, hyperglycemiaCriticalMorning, hyperglycemiaCriticalNight= 0, 0, 0
        hyperglycemiaDay, hyperglycemiaMorning, hyperglycemiaNight= 0, 0, 0
        rangeDay, rangeMorning, rangeNight= 0, 0, 0
        rangeSecondaryDay, rangeSecondaryMorning, rangeSecondaryNight= 0, 0, 0
        hypoglycemia1Day, hypoglycemia1Morning, hypoglycemia1Night= 0, 0, 0
        hypoglycemia2Day, hypoglycemia2Morning, hypoglycemia2Night = 0, 0, 0
        
        data= manualData[(manualData['Date'] == d) & (pd.to_datetime(manualData['Time'],format='%H:%M:%S').dt.time < pd.to_datetime("6:00:00",format='%H:%M:%S').time())].interpolate(limit=10)
        for l in data['Sensor Glucose (mg/dL)']:
            if l > 250:
                hyperglycemiaCriticalNight += 1
            if l <= 250 and l > 180:
                hyperglycemiaNight += 1
            if l <= 180 and l >= 70:
                rangeNight += 1
            if l <= 150 and l >= 70:
                rangeSecondaryNight += 1
            if l < 70 and l >= 54:
                hypoglycemia1Night += 1
            if l < 54:
                hypoglycemia2Night += 1
            cNum += 1
        manualHyperglycemiaCriticalNight += hyperglycemiaCriticalNight/72
        manualHyperglycemiaNight += hyperglycemiaNight/72
        manualRangeNight += rangeNight/72
        manualRangeSecondaryNight += rangeSecondaryNight/72
        manualHypoglycemia1Night += hypoglycemia1Night/72
        manualHypoglycemia2Night += hypoglycemia2Night/72
        totalManualNight += 1
    else:
        pass
manualHyperglycemiaCriticalDay /= totalManual
manualHyperglycemiaCriticalMorning /= totalManualMorning
manualHyperglycemiaCriticalNight /= totalManualNight
manualHyperglycemiaDay /= totalManual
manualHyperglycemiaMorning /= totalManualMorning 
manualHyperglycemiaNight /= totalManualNight
manualRangeDay/= totalManual
manualRangeMorning /= totalManualMorning
manualRangeNight /= totalManualNight
manualRangeSecondaryDay/= totalManual
manualRangeSecondaryMorning /= totalManualMorning 
manualRangeSecondaryNight /= totalManualNight
manualHypoglycemia1Day/= totalManual
manualHypoglycemia1Morning /= totalManualMorning
manualHypoglycemia1Night /= totalManualNight
manualHypoglycemia2Day/= totalManual
manualHypoglycemia2Morning /= totalManualMorning
manualHypoglycemia2Night /= totalManualNight



C:\Users\15713\AppData\Local\Temp\ipykernel_19488\1246815214.py:35: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  data = manualData[(manualData['Date'] == d)].interpolate(limit=10)
C:\Users\15713\AppData\Local\Temp\ipykernel_19488\1246815214.py:35: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  data = manualData[(manualData['Date'] == d)].interpolate(limit=10)
C:\Users\15713\AppData\Local\Temp\ipykernel_19488\1246815214.py:35: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  data = manualData[(manualData['Date'] == d)].interpolate(limit=10)
C:\Users\15713\AppData\Local\Temp\ipykernel_19488\1246815214.py:35: FutureWarni

In [49]:
aDays = autoData["Date"].unique()[::-1]

In [50]:
autoHyperglycemiaCriticalDay, autoHyperglycemiaCriticalMorning, autoHyperglycemiaCriticalNight= 0, 0, 0
autoHyperglycemiaDay, autoHyperglycemiaMorning, autoHyperglycemiaNight= 0, 0, 0
autoRangeDay, autoRangeMorning, autoRangeNight= 0, 0, 0
autoRangeSecondaryDay, autoRangeSecondaryMorning, autoRangeSecondaryNight= 0, 0, 0
autoHypoglycemia1Day, autoHypoglycemia1Morning, autoHypoglycemia1Night= 0, 0, 0
autoHypoglycemia2Day, autoHypoglycemia2Morning, autoHypoglycemia2Night= 0, 0, 0
totalAuto= 0
totalAutoMorning= 0
totalAutoNight= 0

for d in aDays:
    #print(f"Day:{d}, Rows:{len(autoData[(autoData['Date'] == d) & (autoData['Sensor Glucose (mg/dL)'].notna())])}")
    actualNight= len(autoData[(autoData['Date'] == d) & (pd.to_datetime(autoData['Time'],format='%H:%M:%S').dt.time < pd.to_datetime("6:00:00",format='%H:%M:%S').time()) & (autoData['Sensor Glucose (mg/dL)'].notna())])
    totalNight = len(autoData[(autoData['Date'] == d) & (pd.to_datetime(autoData['Time'],format='%H:%M:%S').dt.time < pd.to_datetime("6:00:00",format='%H:%M:%S').time())])
    nightDiff = totalNight - actualNight
    
    actualMorning = len(autoData[(autoData['Date'] == d) & (pd.to_datetime(autoData['Time'],format='%H:%M:%S').dt.time >= pd.to_datetime("6:00:00",format='%H:%M:%S').time()) & (autoData['Sensor Glucose (mg/dL)'].notna())])
    totalMorning = len(autoData[(autoData['Date'] == d) & (pd.to_datetime(autoData['Time'],format='%H:%M:%S').dt.time >= pd.to_datetime("6:00:00",format='%H:%M:%S').time())])
    morningDiff = totalMorning - actualMorning
    
    actualDay = len(autoData[(autoData['Date'] == d) & (autoData['Sensor Glucose (mg/dL)'].notna())])
    totalDay = len(autoData[(autoData['Date'] == d)])
    dayDiff = totalDay - actualDay
    
#     print(f"Day:{d}, Rows:{totalDay}, Diff:{dayDiff}, Morning:{totalMorning}, Diff:{morningDiff}, Night:{totalNight}, Diff:{nightDiff}")
    
    if dayDiff <= 10 or totalDay == 288:
        cNum = 1
        
        hyperglycemiaCriticalDay, hyperglycemiaCriticalMorning, hyperglycemiaCriticalNight= 0, 0, 0
        hyperglycemiaDay, hyperglycemiaMorning, hyperglycemiaNight= 0, 0, 0
        rangeDay, rangeMorning, rangeNight= 0, 0, 0
        rangeSecondaryDay, rangeSecondaryMorning, rangeSecondaryNight= 0, 0, 0
        hypoglycemia1Day, hypoglycemia1Morning, hypoglycemia1Night= 0, 0, 0
        hypoglycemia2Day, hypoglycemia2Morning, hypoglycemia2Night= 0, 0, 0
        
        data= autoData[(autoData['Date'] == d)].interpolate(limit=10)
        for l in data['Sensor Glucose (mg/dL)']:
            if l > 250:
                hyperglycemiaCriticalDay += 1
                if cNum <= 216: hyperglycemiaCriticalMorning += 1  
                else: hyperglycemiaCriticalNight += 1
            if l <= 250 and l > 180:
                hyperglycemiaDay += 1
                if cNum <= 216: hyperglycemiaMorning += 1 
                else: hyperglycemiaNight += 1
            if l <= 180 and l >= 70:
                rangeDay += 1
                if cNum <= 216: rangeMorning += 1
                else: rangeNight += 1
            if l <= 150 and l >= 70:
                rangeSecondaryDay += 1
                if cNum <= 216: rangeSecondaryMorning += 1
                else: rangeSecondaryNight += 1
            if l < 70 and l >= 54:
                hypoglycemia1Day += 1
                if cNum <= 216: hypoglycemia1Morning += 1
                else: hypoglycemia1Night += 1
            if l < 54:
                hypoglycemia2Day += 1
                if cNum <= 216: hypoglycemia2Morning += 1
                else: hypoglycemia2Night += 1
            cNum += 1
        autoHyperglycemiaCriticalDay += hyperglycemiaCriticalDay/288
        autoHyperglycemiaCriticalMorning += hyperglycemiaCriticalMorning/216
        autoHyperglycemiaCriticalNight += hyperglycemiaCriticalNight/72
        autoHyperglycemiaDay += hyperglycemiaDay/288
        autoHyperglycemiaMorning += hyperglycemiaMorning/216
        autoHyperglycemiaNight += hyperglycemiaNight/72
        autoRangeDay += rangeDay/288
        autoRangeMorning += rangeMorning/216
        autoRangeNight += rangeNight/72
        autoRangeSecondaryDay += rangeSecondaryDay/288
        autoRangeSecondaryMorning += rangeSecondaryMorning/216
        autoRangeSecondaryNight += rangeSecondaryNight/72
        autoHypoglycemia1Day += hypoglycemia1Day/288
        autoHypoglycemia1Morning += hypoglycemia1Morning/216
        autoHypoglycemia1Night += hypoglycemia1Night/72
        autoHypoglycemia2Day += hypoglycemia2Day/288
        autoHypoglycemia2Morning += hypoglycemia2Morning/216
        autoHypoglycemia2Night += hypoglycemia2Night/72
        totalAuto += 1
        totalAutoMorning += 1
        totalAutoNight += 1
    elif morningDiff <= 10 or totalMorning == 216:
        cNum = 1
        
        hyperglycemiaCriticalDay, hyperglycemiaCriticalMorning, hyperglycemiaCriticalNight= 0, 0, 0
        hyperglycemiaDay, hyperglycemiaMorning, hyperglycemiaNight= 0, 0, 0
        rangeDay, rangeMorning, rangeNight= 0, 0, 0
        rangeSecondaryDay, rangeSecondaryMorning, rangeSecondaryNight= 0, 0, 0
        hypoglycemia1Day, hypoglycemia1Morning, hypoglycemia1Night= 0, 0, 0
        hypoglycemia2Day, hypoglycemia2Morning, hypoglycemia2Night= 0, 0, 0
        
        data= autoData[(autoData['Date'] == d)& (pd.to_datetime(autoData['Time'],format='%H:%M:%S').dt.time >= pd.to_datetime("6:00:00",format='%H:%M:%S').time())].interpolate(limit=10)
        for l in data['Sensor Glucose (mg/dL)']:
            if l > 250:
                hyperglycemiaCriticalMorning += 1  
            if l <= 250 and l > 180:
                hyperglycemiaMorning += 1 
            if l <= 180 and l >= 70:
                rangeMorning += 1
            if l <= 150 and l >= 70:
                rangeSecondaryMorning += 1
            if l < 70 and l >= 54:
                hypoglycemia1Morning += 1
            if l < 54:
                hypoglycemia2Morning += 1
            cNum += 1
        autoHyperglycemiaCriticalMorning += hyperglycemiaCriticalMorning/216
        autoHyperglycemiaMorning += hyperglycemiaMorning/216
        autoRangeMorning += rangeMorning/216
        autoRangeSecondaryMorning += rangeSecondaryMorning/216
        autoHypoglycemia1Morning += hypoglycemia1Morning/216
        autoHypoglycemia2Morning += hypoglycemia2Morning/216
        totalAutoMorning += 1
    elif nightDiff <= 10 or totalNight == 72:
        cNum = 1
        
        hyperglycemiaCriticalDay, hyperglycemiaCriticalMorning, hyperglycemiaCriticalNight= 0, 0, 0
        hyperglycemiaDay, hyperglycemiaMorning, hyperglycemiaNight= 0, 0, 0
        rangeDay, rangeMorning, rangeNight= 0, 0, 0
        rangeSecondaryDay, rangeSecondaryMorning, rangeSecondaryNight= 0, 0, 0
        hypoglycemia1Day, hypoglycemia1Morning, hypoglycemia1Night= 0, 0, 0
        hypoglycemia2Day, hypoglycemia2Morning, hypoglycemia2Night= 0, 0, 0
        
        data= autoData[(autoData['Date'] == d) & (pd.to_datetime(autoData['Time'],format='%H:%M:%S').dt.time < pd.to_datetime("6:00:00",format='%H:%M:%S').time())].interpolate(limit=10)
        for l in data['Sensor Glucose (mg/dL)']:
            if l > 250:
                hyperglycemiaCriticalNight += 1
            if l <= 250 and l > 180:
                hyperglycemiaNight += 1
            if l <= 180 and l >= 70:
                rangeNight += 1
            if l <= 150 and l >= 70:
                rangeSecondaryNight += 1
            if l < 70 and l >= 54:
                hypoglycemia1Night += 1
            if l < 54:
                hypoglycemia2Night += 1
            cNum += 1
        autoHyperglycemiaCriticalNight += hyperglycemiaCriticalNight/72
        autoHyperglycemiaNight += hyperglycemiaNight/72
        autoRangeNight += rangeNight/72
        autoRangeSecondaryNight += rangeSecondaryNight/72
        autoHypoglycemia1Night += hypoglycemia1Night/72
        autoHypoglycemia2Night += hypoglycemia2Night/72
        totalAutoNight += 1
    else:
        pass
autoHyperglycemiaCriticalDay /= totalAuto
autoHyperglycemiaCriticalMorning /= totalAutoMorning
autoHyperglycemiaCriticalNight /= totalAutoNight
autoHyperglycemiaDay /= totalAuto
autoHyperglycemiaMorning /= totalAutoMorning 
autoHyperglycemiaNight /= totalAutoNight
autoRangeDay/= totalAuto
autoRangeMorning /= totalAutoMorning
autoRangeNight /= totalAutoNight
autoRangeSecondaryDay/= totalAuto
autoRangeSecondaryMorning /= totalAutoMorning 
autoRangeSecondaryNight /= totalAutoNight
autoHypoglycemia1Day/= totalAuto
autoHypoglycemia1Morning /= totalAutoMorning
autoHypoglycemia1Night /= totalAutoNight
autoHypoglycemia2Day/= totalAuto
autoHypoglycemia2Morning /= totalAutoMorning
autoHypoglycemia2Night /= totalAutoNight

C:\Users\15713\AppData\Local\Temp\ipykernel_19488\2308565823.py:37: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  data= autoData[(autoData['Date'] == d)].interpolate(limit=10)
C:\Users\15713\AppData\Local\Temp\ipykernel_19488\2308565823.py:127: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  data= autoData[(autoData['Date'] == d) & (pd.to_datetime(autoData['Time'],format='%H:%M:%S').dt.time < pd.to_datetime("6:00:00",format='%H:%M:%S').time())].interpolate(limit=10)
C:\Users\15713\AppData\Local\Temp\ipykernel_19488\2308565823.py:37: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  data= autoData[(autoData['Date'] == d)]

In [51]:
import csv

In [52]:
f= open('./Result.csv', 'w')
myWriter = csv.writer(f)

myWriter.writerow([manualHyperglycemiaNight * 100, manualHyperglycemiaCriticalNight * 100, manualRangeNight * 100, manualRangeSecondaryNight * 100, manualHypoglycemia1Night * 100, manualHypoglycemia2Night * 100, manualHyperglycemiaMorning * 100, manualHyperglycemiaCriticalMorning * 100, manualRangeMorning * 100, manualRangeSecondaryMorning * 100, manualHypoglycemia1Morning * 100, manualHypoglycemia2Morning * 100, manualHyperglycemiaDay * 100, manualHyperglycemiaCriticalDay * 100, manualRangeDay * 100, manualRangeSecondaryDay * 100, manualHypoglycemia1Day * 100, manualHypoglycemia2Day * 100])
myWriter.writerow([autoHyperglycemiaNight * 100, autoHyperglycemiaCriticalNight * 100, autoRangeNight * 100, autoRangeSecondaryNight * 100, autoHypoglycemia1Night * 100, autoHypoglycemia2Night * 100, autoHyperglycemiaMorning * 100, autoHyperglycemiaCriticalMorning * 100, autoRangeMorning * 100, autoRangeSecondaryMorning * 100, autoHypoglycemia1Morning * 100, autoHypoglycemia2Morning * 100, autoHyperglycemiaDay * 100, autoHyperglycemiaCriticalDay * 100, autoRangeDay * 100, autoRangeSecondaryDay * 100, autoHypoglycemia1Day * 100, autoHypoglycemia2Day * 100])

f.close()

In [ ]:
myWriter.writerow([4.578993055555555,	0.9765625,	18.229166666666664,	13.563368055555555,	0.6293402777777778,	0.0,	25.086805555555557,	10.481770833333332,	39.2578125,	30.598958333333332,	4.578993055555555,	1.8012152777777777,	29.66579861111111,	11.458333333333332,	57.486979166666664,	44.16232638888889,	5.208333333333334,	1.8012152777777777])
myWriter.writerow([2.9185956790123457,	0.45331790123456794,	21.145833333333332,	18.306327160493826,	0.6693672839506173,	0.19675925925925924,	20.8641975308642,	4.799382716049383,	48.95833333333333,	36.84606481481482,	3.9911265432098766,	1.3695987654320987,	23.782793209876544,	5.252700617283951,	70.10416666666667,	55.152391975308646,	4.660493827160494,	1.566358024691358])